In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import astropy.units as u
from sunpy.time import TimeRange
import time
import sys

import datetime as dt
import matplotlib.dates as mdates
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib import gridspec,transforms

mpl.use('nbagg')
%matplotlib inline

In [2]:
sys.path.insert(1, '../../20201013_jupiter/')
from sk import LofarRaw

In [3]:
filename = '../udpoutput/uranus-stokesVectors_0_2020-12-15T20:04:00_19629670898060' #0 is I and 1 is V I assume
frange = [15,30]
sbs = np.arange(76,198)
obs_mode = 3
time_len_mins = 176.
trange = TimeRange(filename.split('_')[-2], time_len_mins*u.min)
xlabel = "Time from {} {}".format(filename.split('_')[-2].split('T')[0], filename.split('_')[-2].split('T')[1])
ylabel = "Frequency (MHz)"
title = filename.split('/')[2]

In [4]:
# read in the data (stokes I or V from udpPM)
raw = LofarRaw(fname=filename, sbs=sbs, obs_mode=obs_mode, frange=frange)
raw.data = raw.data[:,:78] ###the rest of the subbands are empty because of the way ilofar/realta records data into the data lanes

Time to read data: 0.000488s



In [6]:
flimits = np.array([0,78])
freqlimits = raw.sb_to_f(flimits+sbs[0], obs_mode)
xlims = list(map(dt.datetime.fromisoformat, [trange.start.value, trange.end.value]))
xlims = mdates.date2num(xlims)

datasplit = 10
normseconds = 42
norm_splits = int(((trange.minutes.value/datasplit)*60)/normseconds)

tchunks = trange.split(datasplit)
datas = np.array_split(raw.data, datasplit, axis=0)


In [7]:
import ed_flagger as edf

In [8]:
df_filtered = edf.basic_filter(datas[4], 4.) #4 sigma

TypeError: basic_filter() missing 1 required positional argument: 'trigger_threshold'